In [1]:
from astropy import units as u

In [2]:
def convert_and_strip_units(quantity, output_unit=None, digit=10):
    """
    Strips units and return the numerical value.

    Parameters
    ----------
    quantity : int or float or list or None or ~astropy.units.quantity.Quantity
        Numerical quantity. Pass it without including units if default units are intended.
    output_unit : ~astropy.units.core.UnitBase or ~astropy.units.quantity.Quantity
        The default units in which the quantity is to be converted before extracting the value.

    Other Parameters
    ----------------
    
    round: int
        round to that number of digit after conversion. Default ``10``. Set to 
        ``None`` to disable.

    Returns
    -------
    int or float or None or ~numpy.ndarray
        The numerical value extracted from ``quantity``

    Examples
    --------
    
        >>> convert_and_strip_units(4.5 * u.um, u.nm)
        <<< 4500.0

        >>> convert_and_strip_units(200000 * (u.m)**-1, 1/u.cm)
        <<< 2000.0

    Raises
    ------
    TypeError
        Raised when ``quantity`` is a astropy.units quantity and ``output_unit`` is ``None``.
    
    """
    if isinstance(quantity, u.Quantity):
        if output_unit in (u.deg_C, u.imperial.deg_F, u.K):
            quantity = quantity.to_value(output_unit, equivalencies=u.temperature())
        elif isinstance(output_unit, (u.UnitBase, u.Quantity)):
            quantity = quantity.to_value(output_unit, equivalencies=u.spectral())
        else:
            raise TypeError(
                "'output_unit' parameter is not a valid astropy unit: {0}".format(
                    output_unit
                )
            )

        if digit:
            quantity = round(quantity, digit)

    return quantity

In [3]:
def air2vacuum(wv):
    return 1.2*wv

In [4]:
def nm2cm(wv):
    return wv/(10**7)

In [6]:
class Default:
    """ Contains a value
    
    Examples
    --------
    
    ::
    
        a = Default("42")
        isinstance(a, Default)
        >>> True
        
        a.value
        >>> 42
    """

    def __init__(self, value):
        self.value = value


In [7]:
def get_waverange(
    wmin=None,
    wmax=None,
    wunit=Default("cm-1"),   #is none in the actual code because cal
    wavenum_min=None,
    wavenum_max=None,
    wavelength_min=None,
    wavelength_max=None,
    medium="air",
):
    represent_wavelength = False
    represent_wavenum = False
    
    # user did not pass wunit
    if isinstance(wunit, Default):
        wunit = wunit.value
        # user did not pass wmin or wmax
        if wmin is None or wmax is None:
            assert wmin is None and wmax is None
        else:
            assert wmin is not None
            assert wmax is not None
            # user passes wmin/wmax with unit
            if isinstance(wmin, u.Quantity) or isinstance(wmax, u.Quantity):
                assert wmin.unit.is_equivalent(u.m) or wmin.unit.is_equivalent(1 / u.m)
                assert wmax.unit.is_equivalent(u.m) or wmax.unit.is_equivalent(1 / u.m)
                assert wmin.unit.is_equivalent(wmax.unit)
                if wmin.unit.is_equivalent(u.m):
                    represent_wavelength = True
                else:
                    represent_wavenum = True
            # user did not pass wmin/wmax with unit
            else:
                assert not isinstance(wmin, u.Quantity)
                assert not isinstance(wmax, u.Quantity)
                wmin = wmin * u.Unit(wunit)
                wmax = wmax * u.Unit(wunit)
                represent_wavenum = True

    # user passed wunit explicitly
    else:
        # user did not pass wmin/wmax
        if wmin is None or wmax is None:
            raise ValueError("Please enter value of wmin and wmax when passing wunit")
        # user passes wmin/wmax along with wunit
        else:
            assert wmin is not None
            assert wmax is not None
            # wmin/wmax values come with unit
            if isinstance(wmin, u.Quantity) or isinstance(wmax, u.Quantity):
                assert wmin.unit.is_equivalent(u.m) or wmin.unit.is_equivalent(1 / u.m)
                assert wmax.unit.is_equivalent(u.m) or wmax.unit.is_equivalent(1 / u.m)
                assert wmin.unit.is_equivalent(wmax.unit)
                # wmin and wunit are the same units
                if wmin.unit == u.Unit(wunit):
                    if wmin.unit.is_equivalent(u.m):
                        represent_wavelength = True
                    else:
                        represent_wavenum = True
                # wmin and wunit have different units
                else:
                    raise ValueError(
                        "Conflicting units entered for wmin/wmax and wunit"
                    )
            # wmin/wmax do not have units
            else:
                wmin = wmin * u.Unit(wunit)
                wmax = wmax * u.Unit(wunit)
                if wmin.unit.is_equivalent(u.m):
                    represent_wavelength = True
                else:
                    represent_wavenum = True

    if represent_wavenum:
        wmin = convert_and_strip_units(wmin, 1 / u.cm)
        wmax = convert_and_strip_units(wmax, 1 / u.cm)
    if represent_wavelength:
        wmin = convert_and_strip_units(wmin, u.nm)
        wmax = convert_and_strip_units(wmax, u.nm)

    wavelength_min = convert_and_strip_units(wavelength_min, u.nm)
    wavelength_max = convert_and_strip_units(wavelength_max, u.nm)
    wavenum_min = convert_and_strip_units(wavenum_min, 1 / u.cm)
    wavenum_max = convert_and_strip_units(wavenum_max, 1 / u.cm)
    if (
        wmin is None
        and wmax is None
        and wavelength_min is None
        and wavelength_max is None
        and wavenum_min is None
        and wavenum_max is None
    ):
        raise ValueError("Give wavenumber or wavelength")

    # check if only one pair of values has been passed
    w_present = wmin is not None or wmax is not None
    wavenum_present = wavenum_min is not None or wavenum_max is not None
    wavelength_present = wavelength_min is not None or wavelength_max is not None
    if w_present + wavenum_present + wavelength_present >= 2:
        raise ValueError("Cannot pass more than one set of values as input")
    assert medium in ["air", "vacuum"]

    # user has passed valid wmin/wmax with units
    if represent_wavelength:
        assert wmin < wmax
        wavelength_min = wmin
        wavelength_max = wmax
    elif represent_wavenum:
        assert wmin < wmax
        wavenum_min = wmin
        wavenum_max = wmax

    # ... Input is in wavelength:
    if wavenum_min is None and wavenum_max is None:
        assert wavelength_max is not None
        assert wavelength_min is not None
        # Test range is correct:
        assert wavelength_min < wavelength_max

        # In wavelength mode, the propagating medium matters. Convert to
        # calculation medium (vacuum) if needed:
        if medium == "air":
            wavelength_min_vac = air2vacuum(wavelength_min)
            wavelength_max_vac = air2vacuum(wavelength_max)
        else:
            wavelength_min_vac = wavelength_min
            wavelength_max_vac = wavelength_max

        wavenum_min = nm2cm(wavelength_max_vac)
        wavenum_max = nm2cm(wavelength_min_vac)
    # ... or input is in wavenumber:
    else:
        assert wavenum_min is not None
        assert wavenum_max is not None
        # Test range is correct:
        assert wavenum_min < wavenum_max

    return wavenum_min, wavenum_max

## wunit is none

### wmin/wmax is none 

#### wavenumber is passed

In [8]:
get_waverange(wavenum_min = 10, wavenum_max = 20)

(10, 20)

#### wavelength is passed

In [9]:
get_waverange(wavelength_min = 1, wavelength_max = 2, medium="vacuum") #assumes default as nm for wavelength

(2e-07, 1e-07)

#### both are passed

In [10]:
get_waverange(wavenum_min=1000, wavenum_max=2000,
            wavelength_min=1, wavelength_max=2, medium="vacuum")

ValueError: Cannot pass more than one set of values as input

#### none are passed

In [11]:
get_waverange()

ValueError: Give wavenumber or wavelength

### wmin/wmax is value only

#### wavenumber is passed

In [12]:
get_waverange(wavenum_min=1, wavenum_max=2, wmin = 10, wmax = 20)

ValueError: Cannot pass more than one set of values as input

#### wavelength is passed

In [13]:
get_waverange(wmin = 10, wmax = 20, wavelength_min=1, wavelength_max=2)

ValueError: Cannot pass more than one set of values as input

#### both are passed

In [14]:
get_waverange(wmin=1, wmax =2, wavenum_min=10, wavenum_max=20,
              wavelength_min = 100, wavelength_max=200)

ValueError: Cannot pass more than one set of values as input

#### none are passed

In [15]:
get_waverange(wmin=10, wmax=20)

(10.0, 20.0)

### wmin/wmax is value and unit

#### wavenumber is passed

In [16]:
get_waverange(wavenum_min = 10, wavenum_max = 20, wmin = 100 * (1/u.cm), wmax = 200 * (1/u.cm))

ValueError: Cannot pass more than one set of values as input

#### wavelength is passed

In [17]:
get_waverange(wmin = 100 * (1/u.cm), wmax = 200 * (1/u.cm),
             wavelength_min = 10, wavelength_max = 20)

ValueError: Cannot pass more than one set of values as input

#### both are passed

In [18]:
get_waverange(wavenum_min = 10, wavenum_max = 20,
              wmin = 100 * (1/u.cm), wmax = 200 * (1/u.cm),
             wavelength_min = 10, wavelength_max = 20)

ValueError: Cannot pass more than one set of values as input

#### none are passed

In [19]:
get_waverange(wmin = 100 * (1/u.cm), wmax = 200 * (1/u.cm))

(100.0, 200.0)

In [20]:
get_waverange(wmin = 1 * u.cm, wmax = 2 * u.cm, medium = "vacuum")

(2.0, 1.0)

In [ ]:
get_waverange2(wavelength_min = 1*u.cm, wavelength_max=2*u.cm, medium="vacuum")

## WUNIT is NOT None

### wmin wmax is none

#### wavenumber is passed

In [21]:
get_waverange(wavenum_min = 1, wavenum_max = 2 , wunit="cm")

ValueError: Please enter value of wmin and wmax when passing wunit

#### wavelength is passed

In [22]:
get_waverange(wavelength_min = 1, wavelength_max = 2 , wunit="cm")

ValueError: Please enter value of wmin and wmax when passing wunit

#### both are passed

In [23]:
get_waverange(wavenum_min= 1, wavenum_max = 2, wavelength_min = 10, wavelength_max = 20 , wunit="cm")

ValueError: Please enter value of wmin and wmax when passing wunit

#### none are passed

In [24]:
get_waverange(wunit="cm")

ValueError: Please enter value of wmin and wmax when passing wunit

### wmin wmax is value only

#### wavenumber is passed

In [25]:
get_waverange(wmin =1, wmax = 2,wavenum_min = 10, wavenum_max = 20, wunit = "cm")

ValueError: Cannot pass more than one set of values as input

#### wavelength is passed

In [26]:
get_waverange(wmin =1, wmax = 2,wavelength_min = 10, wavelength_max = 20, wunit = "cm")

ValueError: Cannot pass more than one set of values as input

#### both are passed

In [27]:
get_waverange(wmin =1, wmax = 2,wavenum_min = 10, wavenum_max = 20, wavelength_min = 100, wavelength_max=200, wunit = "cm")

ValueError: Cannot pass more than one set of values as input

#### none are passed

In [28]:
get_waverange(wmin =1, wmax = 2, wunit = "cm")

(2.4, 1.2)

In [29]:
get_waverange(wmin=1, wmax=2, wunit = "cm-1")

(1.0, 2.0)

### wmin wmax is both value and unit

#### wavenumber is passed

In [30]:
get_waverange(wmin =1*u.cm, wmax = 2*u.cm,wavenum_min = 10, wavenum_max = 20, wunit = "cm")

ValueError: Cannot pass more than one set of values as input

#### wavelength is passed

In [31]:
get_waverange(wmin= 1*u.cm, wmax = 2*u.cm, wavelength_min = 10, wavelength_max = 20 , wunit="cm")

ValueError: Cannot pass more than one set of values as input

#### both are passed

In [32]:
get_waverange(wmin=1*u.cm, wmax=2*u.cm, wavenum_min= 1, wavenum_max = 2, wavelength_min = 10, wavelength_max = 20 , wunit="cm")

ValueError: Cannot pass more than one set of values as input

#### NONE ARE PASSED

In [33]:
get_waverange(wmin= 1*u.cm, wmax = 2*u.cm, wunit="cm")

(2.4, 1.2)

In [34]:
get_waverange(wmin= 1*(1/u.cm), wmax = 2*(1/u.cm), wunit="cm-1")

(1.0, 2.0)

In [35]:
get_waverange(wmin=1*u.cm, wmax=2*u.cm, wunit="cm")

(2.4, 1.2)

In [36]:
get_waverange(wavelength_min = 1*u.cm, wavelength_max=2*u.cm)

(2.4, 1.2)

In [37]:
get_waverange(wmin=1*u.cm, wmax=2*u.cm, wunit="cm-1")

ValueError: Conflicting units entered for wmin/wmax and wunit

In [38]:
get_waverange(wmin=1*u.cm, wmax=2*u.cm, wunit="m")

ValueError: Conflicting units entered for wmin/wmax and wunit

In [42]:
import pytest
with pytest.raises(ValueError):
    get_waverange(wavenum_min= 1, wavenum_max = 2, wavelength_min = 10, wavelength_max = 20 , wunit="cm")
    get_waverange(wmin = 1 * u.cm, wmax = 2 * u.cm, medium = "vacuum")

In [47]:
get_waverange(wavelength_min=1, wavelength_max=2, medium="vacuum", wunit=Default("cm-1"))

(2e-07, 1e-07)

In [49]:
assert get_waverange2(wavelength_min=1, wavelength_max=2, medium="vacuum", wunit=Default("cm-1")) == (2e-07, 1e-07)

In [46]:
def get_waverange2(
    wmin=None,
    wmax=None,
    wunit=None,
    wavenum_min=None,
    wavenum_max=None,
    wavelength_min=None,
    wavelength_max=None,
    medium="air",
):
    # check input consistency
    # check if atleast one input is given
    if (
        wmin is None
        and wmax is None
        and wavelength_min is None
        and wavelength_max is None
        and wavenum_min is None
        and wavenum_max is None
    ):
        raise ValueError("Give wavenumber or wavelength")

    # check if only one pair of values has been passed
    w_present = wmin is not None and wmax is not None
    wavenum_present = wavenum_min is not None and wavenum_max is not None
    wavelength_present = wavelength_min is not None and wavelength_max is not None
    if w_present + wavenum_present + wavelength_present != 1:
        raise ValueError(
            "Please pass exactly one set of values as input: "
            "choose either wmin/wmax (with astropy.units), "
            "wavenum_min/wavenum_max, or wavelength_min/wavelength_max"
        )

    complete_pairs = 0

    # check for unit consistency

    if not isinstance(wunit, Default):
        if not u.Unit(wunit).is_equivalent(u.m) and not u.Unit(wunit).is_equivalent(
            1 / u.m
        ):
            raise ValueError("Wunit dimensions should be either [length] or 1/[length]")

    if wavelength_min is not None or wavelength_max is not None:
        assert wavelength_min is not None and wavelength_max is not None
        assert wavelength_min < wavelength_max
        if not isinstance(wunit, Default):
            raise ValueError("Please use wmin/wmax when passing wunit")
        if isinstance(wavelength_min, u.Quantity):
            assert isinstance(wavelength_min, u.Quantity) and isinstance(
                wavelength_max, u.Quantity
            )
            assert wavelength_min.unit.is_equivalent(u.m)
            assert wavelength_max.unit.is_equivalent(u.m)

    if wavenum_min is not None or wavenum_max is not None:
        assert wavenum_min is not None and wavenum_max is not None
        assert wavenum_min < wavenum_max
        if not isinstance(wunit, Default):
            raise ValueError("Please use wmin/wmax when passing wunit")
        if isinstance(wavenum_min, u.Quantity) or isinstance(wavenum_max, u.Quantity):
            assert isinstance(wavenum_min, u.Quantity) and isinstance(
                wavenum_max, u.Quantity
            )
            assert wavenum_min.unit.is_equivalent(1 / u.cm)
            assert wavenum_max.unit.is_equivalent(1 / u.cm)

    if isinstance(wmin, u.Quantity) or isinstance(wmax, u.Quantity):
        assert wmin is not None and wmax is not None
        assert isinstance(wmin, u.Quantity) and isinstance(wmax, u.Quantity)
        assert wmin.unit.is_equivalent(u.m) or wmin.unit.is_equivalent(1 / u.m)
        assert wmax.unit.is_equivalent(u.m) or wmax.unit.is_equivalent(1 / u.m)
        assert wmin.unit.is_equivalent(wmax.unit)
        if not isinstance(wunit, Default):
            if not wmin.unit.is_equivalent(u.Unit(wunit)):
                raise ValueError("Conflicting units passed for wmin/wmax and wunit")
            # Should I keep this in?
            # Deals with cases like: calc_spectrum(wmin=10*u.cm, wmax=1.u.m, wunit="cm")
    #             else:
    #                 if wmin.unit != wmax.unit and (wmin.unit != u.Unit(wunit.value) or wmax.unit != u.Unit(wunit.value)):
    #                     raise Warning("Ambiguous units passed, ignoring wunit")

    if wmin is not None and wmax is not None:
        if isinstance(wmin, u.Quantity) or isinstance(wmax, u.Quantity):
            if wmin.unit.is_equivalent(u.m):
                wavelength_min = wmin
                wavelength_max = wmax
            else:
                wavenum_min = wmin
                wavenum_max = wmax
        else:
            if isinstance(wunit, Default):
                wavenum_min = wmin * u.Unit(wunit.value)
                wavenum_max = wmax * u.Unit(wunit.value)
            else:
                if u.Unit(wunit).is_equivalent(u.m):
                    wavelength_min = wmin * u.Unit(wunit)
                    wavelength_max = wmax * u.Unit(wunit)
                else:
                    wavenum_min = wmin * u.Unit(wunit)
                    wavenum_max = wmax * u.Unit(wunit)

    if wavenum_min is not None or wavenum_max is not None:
        wavenum_min = convert_and_strip_units(wavenum_min, 1 / u.cm)
        wavenum_max = convert_and_strip_units(wavenum_max, 1 / u.cm)

    if wavelength_min is not None or wavelength_max is not None:
        assert medium in ["air", "vacuum"]
        wavelength_min = convert_and_strip_units(wavelength_min, u.nm)
        wavelength_max = convert_and_strip_units(wavelength_max, u.nm)
        if medium == "air":
            wavelength_min_vac = air2vacuum(wavelength_min)
            wavelength_max_vac = air2vacuum(wavelength_max)
        else:
            wavelength_min_vac = wavelength_min
            wavelength_max_vac = wavelength_max

        wavenum_min = nm2cm(wavelength_max_vac)
        wavenum_max = nm2cm(wavelength_min_vac)

    return wavenum_min, wavenum_max


In [50]:
get_waverange(wavelength_min=1, wavelength_max=2, medium="vacuum", wunit=Default("cm-1"))

(2e-07, 1e-07)

In [53]:
assert get_waverange(wavelength_min=1, wavelength_max=2, medium="vacuum", wunit=Default("cm-1")) == (0.0000002, 0.0000001)
